In [0]:
# The code was removed by Watson Studio for sharing.

In [0]:
#dd-ignore

!pip install --user decision-optimization-client


In [0]:
#dd-ignore

from decision_optimization_client import *

# Creates a client...
# If you want to be able to call solve() on the client, you have to provide your API Key
# client = Client(pc=pc, apikey='IAM_APIKEY')
client = Client(pc=pc)


In [0]:
#dd-ignore

#Get 'sales_production_planning' experiment...
do_experiment = client.get_experiment(name="sales_production_planning")

#Get scenario 'Baseline'...
scenario = do_experiment.get_scenario(name="Baseline")

#Load all input data as a map { data_name: data_frame }
inputs = scenario.get_tables_data(category='input')
# This will hold all outputs as a map { data_name: data_frame }
outputs = {}

# we use a lock to access ``outputs``. This allows solves() to
# be aborted without race condition in data writting
import threading
output_lock = threading.Lock()



# htk
# Sales and Operations Planning - Model Building
This notebook is an example of how decision optimization can help sales and operations planners to execute the most profitable supply chain strategy while balancing supply and demand. First, introduction and problem description are presented. We then provide the mathematical programming formulation. And following the data understanding and preparation details, mathematical model written in Python is presented. Finally, we conclude the notebook with discussion of results. 

Notebook requires the Commercial Edition of CPLEX engines to solve the optimization model, which is included in the Default Python 3.6 XS + DO in Watson Studio. 
**Contents**

- [Introduction](#Introduction)
- [Problem-Description](#Problem-Description)
- [Formulation](#Formulation)
- [Data Preparation](#Data-Preparation)
    - [Data Input](#Data-Input)
    - [Pre-processing](#Pre-processing)
- [Decision Variables](#Decision-Variables)
    - [Model Instance](#Model-Instance)
    - [Production Plan](#Production-Plan)
    - [Product Allocation](#Product-Allocation)
    - [Sales](#Sales)
    - [Inventory](#Inventory)
- [Constraints](#Constraints)
    - [The total amount of production per month is limited by the capacity of the plant](#The-total-amount-of-production-per-month-is-limited-by-the-capacity-of-the-plant)
    - [Plants cannot produce more than their product-based capacity](#Plants-cannot-produce-more-than-their-product-based-capacity)
    - [Production equals allocation](#Production-equals-allocation)
    - [Executed sales are at least the firm sales](#Executed-sales-are-at-least-the-firm-sales)
    - [Executed sales are at most the planned demand](#Executed-sales-are-at-most-the-planned-demand)
    - [Inject the initial inventory to the decision variable inventory](#Inject-the-initial-inventory-to-the-decision-variable-inventory)
    - [Starting Inventory + Allocation = Executed Sales + Ending Inventory](#Starting-Inventory-+-Allocation-=-Executed-Sales-+-Ending-Inventory)
    - [Ending Inventory = Starting Inventory of the previous month (no inventory at the end of last month)](#Ending-Inventory-=-Starting-Inventory-of-the-previous-month-(no-inventory-at-the-end-of-last-month))
- [KPIs and Objectives](#KPIs-and-Objectives)
- [Model Solving](#Model-Solving)
- [Solution Handling](#Solution-Handling)
- [Solution Review](#Solution-Review)
## Introduction
Manufacturing companies, producing a wide variety of products for global  markets, must periodically provide production plans for their geographically distributed plants. Production capacity is matched up with forecasted sales demand. The critical decision is how to best allocate production among different facilities to meet worldwide market demands; and as a result high-level production plans will be generated.

Costs and service levels are two main objectives of interest in a typical supply chain. Both objectives can be better optimized by collaborative decision-making and optimization models. Lack of integration in production and distribution operations yields to substantial inefficiencies and, consequently, poor total system performance. And, many companies prefer to hold limited inventory in the supply chain. Therefore, coordinating production and distribution operations becomes more crucial for satisfying on-time delivery requirements without intermediate storage. 

In such a situation, sourcing decisions to reduce transportation costs are highly affected by production capacities and costs. Motivated by the fact that an increasing number of companies are now adopting advanced analytical models, we study the sales and operations planning problem of optimizing customer service levels and total costs in an integrated production and distribution environment. 

Because generally there are alternative production facilities with limited capacities, overall supply chain costs together with capacities and capabilities are main drivers for sales and operations planning decisions.

Unique challenges can be summarized as: 
- Fulfillment vs cost trade-offs
- Integrated production and distribution decisions
- Need to respond faster to market shifts and analyze what-if scenarios

And expected benefits include: 
- Better fulfillment
- Reduced costs
- Quick  responses to
    - Supply disruption
    - Demand fluctuations (e.g.  new promotions, popular products)
## Problem Description
The problem considered in this notebook consists of production planning of customer orders(motorcycles) distributed in different geographies (markets). There exist a number of manufacturing facilities with different capacities and capabilities. Unit production cost and unit transportation cost occur depending on the sourcing facility of the demand. Planning is performed on a monthly basis. Orders (or demand forecasts) are received by manufacturer and processed by eligible facilities. Problem can be defined as deciding on how much to produce in each month and in each facility. 

Firm sales (which are already agreed to be satisified) are also considered and model needs to ensure that firm sales are satisfied regardless of profit. Model can decide on fully, partially or not at all satisfying the remaining demand based on profitability and availaible capacity. Inventory build-up (producing an order at an earlier month) is allowed but back-orders (fulfilling an order at a later month) are not allowed. The objective is to maximize total profit which is total revenue minus production and transportation costs. 

As summary, company manufactures motorcycles for global markets. And it has to provide monthly plans for its geographically distributed plants. This example provides a look at how this manufacturer handles their monthly planning process, in which production capacity is matched up with forecasted sales demand.
## Formulation
In this section, we formulate the problem in a mathematical way, which will help us to better understand the problem, to make decisions about how to prepare the datasets and how to construct the code in a more efficient way. 
### Notation 
**Time** The sales and production planning takes place on a monthly basis. Let $T := \bigcup\limits_{t}t$ be the set of all months. 

**Products** Let $P := \bigcup\limits_{p}p$ be the set of all products. 

**Plants** Let $L := \bigcup\limits_{l}l$ be the set of plants [*Plants*]. The function $c: L\times T \rightarrow \mathbb{N}$ expresses the capacity of a plant on a monthly level [*PlantMonthCapacities*]. For convenience, we write $c_{lt}$ as value of the function.  

**Plants-Products** Define $\Lambda$ as the set of pairs $(l, p)$, if the product $p \in P$ could be produced in the plant $l \in L$. The cost of producing a product $p \in P$ in the plant $l \in L$ is reflected on the function $r: \Lambda \rightarrow \mathbb{N}$  [*PlantProductCosts*]. Compared to the plant level capacity, there is also a capacity on the product level. Define  $\tilde{c}: \Lambda\times T \rightarrow \mathbb{N}$, which specifies the capacity on a plant-product-month level  [*PlantProductMonthCapacities*].

**Markets** Let $M := \bigcup\limits_{m}m$ be the set of markets.

**Markets-Products** We call the maximum capacity of a market demand in this formulation. In other words, the sales of a specific product in one market could not exceed its demand. If there is demand for product $p \in P$ in the market $m \in M$ in the month $t \in T$, then we say $(p, m, t) \in \Pi$. The function $d: \Pi \rightarrow \mathbb{N}$ defines the demand on a product-market-month level [*Demands*]. One portion of the demand is called firm sales, which refer to planned orders that can be frozen in quantity and time. Define $f: \Pi \rightarrow \mathbb{N}$ as the firm sales [*FirmSales*].
The function $e: P \times M \times T \rightarrow$ defines the sales price for a product $p \in P$, in the market $m \in M$, for the month $t \in T$ [*SalesPrice*]. Define $i: P \times M \rightarrow \mathbb{N}$, which represents the initial inventory of the product $p \in P$ in the market $m \in M$ [*InitialInventories*]. 

**Transportation Cost**  The mapping $h: \Lambda \times M \rightarrow \mathbb{N}$ represents the transportation cost of the product $p\in P$, which is produced in the plant $l, (l,p) \in \Lambda$ and is allocated to the market $m \in M$ [*Transportation*].

### Decision Variables
**Production** Let $O_{lpt}$ represent the production of the product $p \in P$ in the plant $l \in L$ for the month $t \in T$, where $(l, p) \in \Gamma$.

**Allocation** The variable $A_{lpmt}$ represents the number of units of product $p \in P$, which are produced in the plant $l \in L, (l,p) \in \Lambda$ and allocated to the market $m\in M$ in the month $t \in T$.

**Executed Sales** The variable $S_{pmt}$ maps the product $p$, the market $m$ and the month $t$ to the executed sales, where $(p, m, t) \in \Pi$. 

**Inventory** The inventory at the beginning of the month $t \in T$ for the product $p\in P$ in the market $m\in M$ is called $V_{pmt}$. The same is also defined for the end of the month, $W_{pmt}$, which only exists for result presentation purpose. 

### MIP Formulation
#### 1. The total amount of production per month is limited by the capacity of the plant.
$
\begin{equation}
\sum\limits_{p: (p,l) \in \Gamma} O_{lpt} \leq c_{lt}, \qquad \forall l \in L, t \in T.
\end{equation}
$

#### 2. Plants (for each product and month) cannot produce more than their product-based capacity.
$
\begin{equation}
O_{lpt} \leq \tilde{c}_{lpt}, \qquad \forall (l,p) \in \Lambda, t \in T.
\end{equation}
$

#### 3. Production equals allocation.
$
\begin{equation}
O_{lpt} = \sum\limits_{m\in M} A_{lpmt}, \qquad \forall (l,p) \in \Lambda, t \in T.
\end{equation}
$

#### 4. Executed Sales are at least the firm sales.
$
\begin{equation}
S_{pmt} \geq f_{pmt}, \qquad \forall (p,m,t) \in \Pi.
\end{equation}
$

#### 5. Executed Sales are at most the planned demand.
$
\begin{equation}
S_{pmt} \leq d_{pmt}, \qquad \forall (p,m,t) \in \Pi.
\end{equation}
$

#### 6. Inject the initial inventory to the first time period of the decision variable starting inventory
$
\begin{equation}
V_{pmt} = i_{pm}, \qquad \forall p \in P, m \in M, \textit{if } t=\min\{t, t\in T\}.
\end{equation}
$

#### 7. Starting Inventory + Allocation = Executed Sales + Ending Inventory.
$
\begin{equation}
V_{pmt} + \sum \limits_{l \in L: (l,p)\in \Lambda} A_{lpmt}= S_{pmt} + W_{pmt}, \qquad \forall p \in P, m \in M, t\in T.
\end{equation}
$

#### 8. Ending Inventory = Starting Inventory of the next month (no inventory at the end of last month).
$
\begin{align}
W_{pmt} &= V_{pmt^{+}} , \qquad &\forall p \in P, m \in M, t\in T\setminus\max\{t, t\in T\}, \\
W_{pmt} &= 0, \qquad &\textit{if } t = \max\{t, t\in T\}.
\end{align}
$

### Objective 
**Maximize the total profit** The sole objective of this project is to maximize the total profit, which is in this case the difference between the sales price and the cost (production cost and transportation cost). This statement holds since we assume that all the other costs are relatively low, such that we do not need to consider for production and sales planning. Thus, this problem could be translated into the following expression.  
$
\begin{align*}
\max \bigg\{ \sum\limits_{(p,m,t)\in\Pi} S_{pmt}~e_{pmt} - \sum\limits_{(l,p) \in \Lambda} r_{lp} \sum\limits_{t\in T} O_{lpt} - \sum\limits_{(l,p) \in \Lambda, m\in M} h_{lpm} \sum\limits_{t\in T} A_{lpmt}  \bigg\}. 
\end{align*}
$

In [0]:
import pandas as pd
pd.set_option('display.max_columns', None)

Now we need to unify the column names and set indexes of the dataframes.

In [0]:
month = inputs["Months"].rename(columns={"name":"month"}).set_index(["month"], verify_integrity=True)
product = inputs["Products"].rename(columns={"name":"product"}).set_index(["product"], verify_integrity=True)
plant_month_capacity = inputs["PlantMonthCapacities"].rename(columns={"value":"plant_month_capacity"}).set_index(["plant", "month"], verify_integrity=True)
plant_product_month_capacity = inputs["PlantProductMonthCapacities"].rename(columns={"value":"plant_product_month_capacity"}).set_index(["plant", "product", "month"], verify_integrity=True)
plant_product_cost = inputs["PlantProductCosts"].rename(columns={"value":"plant_product_cost"}).set_index(["plant", "product"], verify_integrity=True)
market = inputs["Markets"].rename(columns={"name":"market"}).set_index(["market"], verify_integrity=True)
initial_inventory = inputs["InitialInventories"].rename(columns={"value":"initial_inventory"}).set_index(["product", "market"], verify_integrity=True)
transportation_cost = inputs["Transportation"].rename(columns={"name":"market", "value":"transportation_cost"}).set_index(["plant", "product", "market"], verify_integrity=True)
demand = inputs["Demands"].rename(columns={"value":"demand"}).set_index(["product", "market", "month"], verify_integrity=True)
firm_sales = inputs["FirmSales"].rename(columns={"value":"firm_sales"}).set_index(["product", "market", "month"], verify_integrity=True)
sales_price = inputs["SalesPrice"].rename(columns={"value":"sales_price"}).set_index(["product", "market", "month"], verify_integrity=True)

## Pre-processing
The following tables would help us to create decision variables.

In [0]:
market_overview = demand.merge(sales_price, how='left', left_index=True, right_index=True).merge(firm_sales, how='left', left_index=True, right_index=True).fillna(0)
market_overview.head(3)

Create a dataset for starting and ending inventory.

In [0]:
product_idx = product.index
market_idx = market.index
month_idx = month.index
idx = pd.MultiIndex.from_product([product_idx, market_idx, month_idx], names=['product', 'market', 'month'])
inventory = pd.DataFrame(index=idx)
inventory.head(3)

Enrich the transportation cost table with a time axis.

In [0]:
idx = plant_product_month_capacity.index
transportation = pd.DataFrame(index=idx).reset_index()
transportation = pd.concat([transportation]*len(market_idx))
transportation["market"] = market_idx.values.repeat(len(plant_product_month_capacity))
#transportation = transportation.set_index(["plant", "product", "market", "month"]).merge(transportation_cost, left_index=True, right_index=True, how="left").drop(columns=["region"])
transportation = transportation.merge(transportation_cost.reset_index(), how="left", on=["plant", "product", "market"]).drop(columns=["region"]).set_index(["plant", "product", "market", "month"])
transportation.head(3)

Some additional information

In [0]:
firstmonthName = month[month["id"]==min(month["id"])].index.values[0]
lastmonthName = month[month["id"]==max(month["id"])].index.values[0]

Add the production cost to the plant$*$product$*$month capacity table.

In [0]:
#plant_product_month_capacity = plant_product_month_capacity.merge(plant_product_cost, how="left", left_index=True, right_index=True)
plant_product_month_capacity = plant_product_month_capacity.reset_index().merge(plant_product_cost.reset_index(), how="left", on=["plant","product"]).set_index(["plant","product","month"])
plant_product_month_capacity.head(3)

## Decision variables

In [0]:
from docplex.mp.model import Model
mdl = Model(name='sales_and_operation_planning', log_output=True)

### Production Plan
We first define the variables, which represents the production in every plant on a monthly level.

In [0]:
plant_product_month_capacity["Production"] = pd.Series(
    mdl.integer_var_list(plant_product_month_capacity.index, lb=0, ub=plant_product_month_capacity.plant_product_month_capacity, name="Production"), 
    index=plant_product_month_capacity.index)
plant_product_month_capacity.head(3)

### Product Allocation
After a product is being produced, it will be allocated to a specific market, based on the month and the market. This is reflected on the following variable, `Allocation`. 

In [0]:
transportation["Allocation"] = pd.Series(
    mdl.integer_var_list(transportation.index, lb=0, name="Allocation"), index=transportation.index)
transportation.head(3)

### Sales
As the name `ExecutedSales` implies, this variable expresses the sales amount in a specific market in a month. 

In [0]:
market_overview["ExecutedSales"] = pd.Series(
    mdl.integer_var_list(market_overview.index, lb=market_overview.firm_sales, ub=market_overview.demand, name="ExecutedSales"), index=market_overview.index)
market_overview.head(3)

### Inventory
`StartingInventory` defines the starting inventory of every product on the market and month-start level, whereas `EndingInventory` marks the inventory for the end of the month. Note that the latter variable is defined only for result analysis purpose. 

In [0]:
inventory["StartingInventory"] = pd.Series(
    mdl.integer_var_list(inventory.index, lb=0, name="StartingInventory"), index=inventory.index)
#inventory["EndingInventory"] = inventory.groupby(["product", "market"])["StartingInventory"].shift(-1, fill_value=0)
inventory["EndingInventory"] = inventory.groupby(["product", "market"])["StartingInventory"].shift(-1).fillna(0)
inventory.head(3)

## Constraints
### Inject the initial inventory to the decision variable inventory

In [0]:
for pr, mr in initial_inventory.index.values:
    inventory_first = inventory.at[(pr,mr,firstmonthName), "StartingInventory"]
    mdl.add_constraint(inventory_first == initial_inventory.at[(pr,mr), "initial_inventory"], ctname="inventory_initialization_{}_{}".format(pr,mr))

### The total amount of production per month is limited by the capacity of the plant

In [0]:
for pl_mn, group in plant_product_month_capacity.groupby(["plant", "month"]):
    month_capacity = plant_month_capacity.at[pl_mn,"plant_month_capacity"]
    mdl.add_constraint(mdl.sum(group.Production) <= month_capacity, ctname="monthly_totalproduction_capacity_{}".format(pl_mn))

### Plants cannot produce more than their product-based capacity
This constraint is already included in the definition of the decision variable `Production` as the upper bound. 
### Production equals Allocation

In [0]:
for pl_pr_mn, group in transportation.groupby(["plant", "product", "month"]):
    production = plant_product_month_capacity.at[pl_pr_mn, "Production"]
    mdl.add_constraint(mdl.sum(group.Allocation) == production, ctname="monthly_production_alignment_{}".format(pl_pr_mn))

### Executed sales are at least the firm sales
This constraint is also included in the definition of the decision variable `ExecutedSales` as the lower bound.
### Executed sales are at most the planned demand
Demand is apparently the upper bound in the definition of `ExecutedSales`.
### Starting Inventory + Allocation = Executed Sales + Ending Inventory

In [0]:
for pr, mr, mn in inventory.index.values:
    inventory_current = inventory.loc[(pr,mr,mn)]
    mdl.add_constraint(
        inventory_current.StartingInventory + 
        #mdl.sum(transportation.query("product==@pr & market==@mr & month==@mn").Allocation) 
        mdl.sum(transportation.loc[pd.IndexSlice[:,pr,mr,mn], "Allocation"])==
        mdl.sum(market_overview.at[(pr2, mr2, mn2),"ExecutedSales"] for (pr2, mr2, mn2) in market_overview.index.values if pr2==pr and mr2==mr and mn2==mn) +
        inventory_current.EndingInventory, ctname="balancing_{}_{}_{}".format(pr, mr, mn))

### Ending Inventory = Starting Inventory of the previous month (no inventory at the end of last month)
This constraint is considered in the definition of the column `EndingInventory`. In other words, we just created this column, but with the decision variable `StartingInventory`, by taking advantage of this formulation.
## KPIs and Objectives
There are three KPIs we would like to consider in this simplified business case, the production cost, transporation cost as well as the revenue.

In [0]:
production_cost_kpi = mdl.sum(plant_product_month_capacity.Production*plant_product_month_capacity.plant_product_cost)
mdl.add_kpi(production_cost_kpi, "Total_production_cost")

transportation_cost_kpi = mdl.sum(transportation.Allocation*transportation.transportation_cost)
mdl.add_kpi(transportation_cost_kpi, "Total_transportation_cost")

revenue_kpi = mdl.sum(market_overview.sales_price*market_overview.ExecutedSales)
mdl.add_kpi(revenue_kpi, "Revenue")

mdl.maximize(revenue_kpi - transportation_cost_kpi - production_cost_kpi)

## Model Solving
- The log_output shows cplex log
- Always add a timelimit in case of unexpected very long solves

In [0]:
msol = mdl.solve(log_output=True, cplex_parameters = {'timelimit':60})
if msol is not None:
    mdl.report()

## Solution Handling
Now let us save the solution values in the dictionary `outputs`, which then could be used in the scenario to generate some visualizations for model understanding.

In [0]:
plant_product_month_capacity["production"] = [production.solution_value for production in plant_product_month_capacity.Production]
production_output = plant_product_month_capacity.reset_index().drop(columns=["Production","plant_product_cost", "plant_product_month_capacity","product","plant"]).groupby(["month"]).sum().reset_index()
plant_capacity = plant_month_capacity.groupby(["month"]).sum().reset_index()
production_output = production_output.merge(plant_capacity, on=["month"], how="left")
production_output["utilization"] = production_output.production/production_output.plant_month_capacity
production_output["month"] = pd.to_datetime(production_output["month"], format='%b-%y')
production_output.head()

In [0]:
market_overview["executedsales"] = [executedsales.solution_value for executedsales in market_overview.ExecutedSales]
executedsales_output = market_overview.drop(columns=["ExecutedSales","firm_sales","sales_price"]).groupby(["month"]).sum().reset_index()
executedsales_output["month"] = pd.to_datetime(executedsales_output["month"], format="%b-%y")
executedsales_output["satisfied_demand"] = executedsales_output.executedsales/executedsales_output.demand
executedsales_output.head()

In [0]:
outputs['production_output'] = production_output
outputs['executedsales_output'] = executedsales_output